In [12]:
import pandas as pd
import os
import sqlite3 as sql3

In [13]:
currentdir = os.getcwd()
db_path = f"{currentdir}\\S30 ETL Assignment.db"
# print(db_path)

In [14]:
def tables_extract(db_path):
    conn = sql3.connect(db_path)
    dfs = {}

    query = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
    tables = pd.read_sql(query, conn)['name'].tolist()

    for table in tables:
        dfs[table] = pd.read_sql(f"SELECT * FROM {table}", conn)

    conn.close()
    
    return dfs

In [15]:
dfs = tables_extract(db_path)

In [16]:
dfs

{'sales':      sales_id  customer_id
 0           1           10
 1           2           85
 2           3           42
 3           4           93
 4           5           75
 ..        ...          ...
 495       496           22
 496       497           68
 497       498           77
 498       499           10
 499       500           31
 
 [500 rows x 2 columns],
 'orders':       order_id  sales_id  item_id  quantity
 0            1         1        1       2.0
 1            2         1        2       NaN
 2            3         1        3       1.0
 3            4         2        1       1.0
 4            5         2        2       1.0
 ...        ...       ...      ...       ...
 1495      1496       499        2       NaN
 1496      1497       499        3       2.0
 1497      1498       500        1       1.0
 1498      1499       500        2       2.0
 1499      1500       500        3       4.0
 
 [1500 rows x 4 columns],
 'items':    item_id item_name
 0        1        

In [17]:
# Create filtered customer_sales subset
df_customers = dfs['customers'][(dfs['customers']['age'] >= 18) & (dfs['customers']['age'] <= 35)]
df_customer_sales = df_customers.merge(dfs['sales'], on='customer_id', how='inner')

# Create orders_items subset
df_orders_items = dfs['orders'].merge(dfs['items'], on='item_id', how='inner')

# Process df_orders_items
df_orders_items = df_orders_items[df_orders_items['quantity'] > 0]
df_orders_items['quantity'] = df_orders_items['quantity'].astype('int64')

# Merge customer_sales and df_orders_items ON sales_id
df_merged_all = df_customer_sales.merge(df_orders_items, on='sales_id', how='inner')

# Aggregate
df_final = (df_merged_all.groupby(['customer_id', 'age', 'item_name'])
                    .agg(order_quantity=('quantity', 'sum'))
                    .reset_index()
                    .query('order_quantity > 0')
                    .sort_values(by=['customer_id', 'order_quantity'], ascending=[True, False]))

In [ ]:
df_final

In [18]:
# df_final.to_csv('test_pandas.csv',sep=';',index=False)
df_final.to_csv('test_pandas.csv',sep=',',index=False)